<a href="https://colab.research.google.com/github/Saumyadeepm/FreeCodeCamp-Neural-Network-SMS-Text-Classifier/blob/main/FreeCodeCamp_Neural_Network_SMS_Text_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Neural Network SMS Text Classifier***

1. **Problem Statement**
    - *The objective is to develop a neural network-based classifier for SMS text messages. The task involves loading and preprocessing SMS datasets, training a neural network model, and evaluating its performance in classifying text messages as "ham" or "spam".*
    
1. **Objective**
  - *To implement a robust neural network model capable of accurately classifying SMS text messages as either "ham" (non-spam) or "spam". This involves preprocessing text data, training the neural network model, and assessing its performance using validation and test datasets.*

1. **Tasks**
  - *Load the SMS training and validation datasets from TSV files.*
  - *Preprocess the text data by converting categorical variables to numerical representations.*
  - *Construct and train a neural network model using appropriate layers and activation functions.*
  *Evaluate the model's performance using validation and test datasets, measuring metrics such as accuracy and loss.*

1. **Deliverables**
  - *A trained neural network model capable of accurately classifying SMS text messages.*
  - *Documentation outlining the preprocessing steps and neural network architecture.*
  - *Evaluation report detailing the model's performance metrics on validation and test datasets.*

1. **Success Criteria**
  - *The project will be considered successful if the trained model achieves high accuracy in classifying SMS messages as "ham" or "spam" on the validation and test datasets.*

## Import Libraries

In [ ]:
# import libraries
import pandas as pd
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense
from tensorflow.keras.callbacks import EarlyStopping

## Data Retrieval for SMS Text Classification

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-03-21 06:41:10--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.02s   

2024-03-21 06:41:10 (14.6 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2024-03-21 06:41:10--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[====

## Defining Column Names

In [ ]:
names = ["class", "message"]


## Loading Train Data

In [ ]:
train_file = pd.read_csv(train_file_path, sep='\t', names=names)
train_file

,class,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...



## Loading Test Data

In [ ]:
test_file = pd.read_csv(test_file_path, sep='\t', names=names)
test_file

,class,message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


## Preparing Training and Testing Data

In [ ]:
train_message = train_file["message"].values.tolist()
train_label = np.array([0 if x=="ham" else 1 for x in train_file['class'].values.tolist()])
test_message = test_file["message"].values.tolist()
test_label = np.array([0 if x=="ham" else 1 for x in test_file['class'].values.tolist()])

## Building Vocabulary Dictionary

In [ ]:
vocabulary_dict = {}
for messgae in train_message:
  for vocabulary in messgae.split():
    if vocabulary not in vocabulary_dict:
      vocabulary_dict[vocabulary] = 1
    else:
      vocabulary_dict[vocabulary] += 1

## Determining Vocabulary Size and Maximum Sequence Length

In [ ]:
VOCAB_SIZE = len(vocabulary_dict)
MAX_LENGTH = len(max(train_message, key=lambda p: len(p.split())).split())

## Text Data Encoding and Padding

In [ ]:
encoded_train_message = [one_hot(d, VOCAB_SIZE) for d in train_message]
padded_train_message = pad_sequences(encoded_train_message, maxlen=MAX_LENGTH, padding='post')
encoded_test_message = [one_hot(d, VOCAB_SIZE) for d in test_message]
padded_test_message = pad_sequences(encoded_test_message, maxlen=MAX_LENGTH, padding='post')

## Model Training Using LSTM and Convolutional Layers

In [ ]:
from tensorflow.keras.layers import LSTM, Bidirectional, SpatialDropout1D, Conv1D, GlobalMaxPooling1D

# Define the model architecture
model = Sequential()
embedding_layer = Embedding(VOCAB_SIZE, 100, input_length=MAX_LENGTH)
model.add(embedding_layer)
model.add(SpatialDropout1D(0.2))  # Spatial dropout for regularization
model.add(Bidirectional(LSTM(64, return_sequences=True)))  # Bidirectional LSTM for capturing context
model.add(Conv1D(128, 5, activation='relu'))  # Convolutional layer for capturing local patterns
model.add(GlobalMaxPooling1D())  # Global max pooling to reduce dimensionality
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
monitor = EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=10, verbose=1, mode='max', restore_best_weights=True)
model.fit(padded_train_message, train_label, validation_data=(padded_test_message, test_label), callbacks=[monitor], epochs=1000, verbose=2)

Epoch 1/1000
131/131 - 26s - loss: 0.2398 - acc: 0.9162 - val_loss: 0.0436 - val_acc: 0.9878 - 26s/epoch - 202ms/step
Epoch 2/1000
131/131 - 7s - loss: 0.0253 - acc: 0.9933 - val_loss: 0.0516 - val_acc: 0.9885 - 7s/epoch - 53ms/step
Epoch 3/1000
131/131 - 5s - loss: 0.0100 - acc: 0.9974 - val_loss: 0.0411 - val_acc: 0.9892 - 5s/epoch - 41ms/step
Epoch 4/1000
131/131 - 5s - loss: 0.0017 - acc: 0.9998 - val_loss: 0.0483 - val_acc: 0.9907 - 5s/epoch - 36ms/step
Epoch 5/1000
131/131 - 3s - loss: 4.2811e-04 - acc: 0.9998 - val_loss: 0.0556 - val_acc: 0.9892 - 3s/epoch - 22ms/step
Epoch 6/1000
131/131 - 3s - loss: 4.0346e-05 - acc: 1.0000 - val_loss: 0.0597 - val_acc: 0.9892 - 3s/epoch - 21ms/step
Epoch 7/1000
131/131 - 4s - loss: 2.9161e-05 - acc: 1.0000 - val_loss: 0.0621 - val_acc: 0.9892 - 4s/epoch - 28ms/step
Epoch 8/1000
131/131 - 3s - loss: 2.2556e-05 - acc: 1.0000 - val_loss: 0.0635 - val_acc: 0.9899 - 3s/epoch - 19ms/step
Epoch 9/1000
131/131 - 3s - loss: 1.0638e-05 - acc: 1.0000 - 

## Message Classification Prediction Function

In [ ]:
# function to predict messages based on model
def predict_message(pred_text):
  class_dict = {
      0 : "ham",
      1 : "spam",
      }
  encoded_message = [one_hot(pred_text, VOCAB_SIZE)]
  padded_message = pad_sequences(encoded_message, maxlen=MAX_LENGTH, padding='post')
  prediction = [model.predict(padded_message)[0][0], class_dict[np.round(model.predict(padded_message)[0][0])]]
  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 22ms/step
[1.6484195e-05, 'ham']



## Testing Message Classification Predictions

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 35ms/step
You passed the challenge. Great job!
